#Import Libraries

In [13]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact, fixed
from panel import widgets
import panel as pn
from trend_search import google_lookup

#Load Variables etc.

In [14]:
pn.extension("plotly")
#pn.extension(comms='ipywidgets')

#Load Data

In [15]:
labour_naics_path = Path("../Employment/characteristics_naics.csv")
business_activity_path = Path("../Business/businesses.csv")
google_trends_inflation_path = Path("../Inflation.csv")

In [16]:
df_labour = pd.read_csv(labour_naics_path)
df_labour['Date'] = pd.to_datetime(df_labour['REF_DATE'], format='%b-%y')
#df_labour['month_year'] = df_labour['Date'].dt.to_period('M')
df_labour['month_year'] = df_labour['Date']
#print(df_labour.describe(include = 'all'))
df_labour = df_labour[['month_year', 'GEO', 'North American Industry Classification System (NAICS)','VALUE']]
df_labour.set_index("month_year",inplace = True, drop = False)
df_labour.rename(mapper={'North American Industry Classification System (NAICS)': 'Industry'}, axis = 1, inplace = True) 

df_business = pd.read_csv(business_activity_path)
df_business['Date'] = pd.to_datetime(df_business['REF_DATE'], format='%b-%y')
#df_business['month_year'] = df_business['Date'].dt.to_period('M')
df_business['month_year'] = df_business['Date']
#print(df_business.describe(include = 'all'))
df_business = df_business[['month_year', 'GEO', 'Industry','Business dynamics measure','VALUE']]
df_business.set_index("month_year",inplace = True, drop = True)

df_inflation_trend = pd.read_csv(google_trends_inflation_path)

In [17]:
df_labour.head()

,month_year,GEO,Industry,VALUE
month_year,,,,
2019-01-01,2019-01-01,Canada,"Total, all industries",18483.9
2019-02-01,2019-02-01,Canada,"Total, all industries",18580.6
2019-03-01,2019-03-01,Canada,"Total, all industries",18586.8
2019-04-01,2019-04-01,Canada,"Total, all industries",18781.6
2019-05-01,2019-05-01,Canada,"Total, all industries",19175.0


In [18]:
df_business.head()

,GEO,Industry,Business dynamics measure,VALUE
month_year,,,,
2019-01-01,Canada,Business sector industries [T004],Active businesses,893957.0
2019-02-01,Canada,Business sector industries [T004],Active businesses,895634.0
2019-03-01,Canada,Business sector industries [T004],Active businesses,901177.0
2019-04-01,Canada,Business sector industries [T004],Active businesses,907709.0
2019-05-01,Canada,Business sector industries [T004],Active businesses,912273.0


In [19]:
df_inflation_trend.head()

,date,Canada,isPartial,Alberta,British Columbia,Saskatchewan,Manitoba,Ontario,Quebec,Newfoundland,Nova Scotia,New Brunswick,Prince Edward Island,Yukon,Northwest Territories,Nunavut
0,2019-01-06,41,False,15,38,0,0,26,38,33,23,0,0,0,0,-1
1,2019-01-13,49,False,44,33,38,39,39,62,33,23,0,0,0,0,-1
2,2019-01-20,55,False,37,42,38,39,46,30,0,88,29,0,0,0,-1
3,2019-01-27,39,False,15,42,19,38,39,36,0,46,32,0,0,0,-1
4,2019-02-03,44,False,50,36,19,0,35,51,32,46,0,0,0,0,-1


#Create two charts

##First, Create helper items for function

In [20]:
df_business["GEO"] = df_business["GEO"].str.split().str.join(' ')

geography_list = list(set(df_business["GEO"]))
geography_list2 = list(set(df_labour["GEO"]))
industry_list = set(df_business["Industry"])
industry_list2 = set(df_labour["Industry"])
industry_list = list(industry_list.intersection(industry_list2))
# print(industry_list)
# print(industry_list2)
# print(geography_list)
# print(geography_list2)
# print(df_business.head())

In [21]:
#Suggestion on stackoverflow to fix some sort of unicode character
df_business["GEO"] = df_business["GEO"].str.split().str.join(' ')

##Second, Create a Function

In [22]:
def create_chart(dataframe1, dataframe2, geography, industry):
    temp = dataframe1.loc[dataframe1["GEO"]==geography]
    temp = temp.loc[temp["Industry"]==industry]
    temp2 = dataframe2.loc[dataframe2["GEO"]==geography]
    temp2 = temp2.loc[temp2["Industry"]==industry]
    temp2 = temp2.loc[temp2["Business dynamics measure"]=="Active businesses"]
    return pn.Row(temp.hvplot.line(x="month_year", y = "VALUE", xlabel = "month_year", ylabel="VALUE", color = "red", title="Labour numbers: " + industry + " " + geography, rot = 90, height = 400, width = 400).opts(shared_axes = False), temp2.hvplot.line(x="month_year", y = "VALUE",xlabel = "month_year", ylabel="VALUE", color = "red", title="Business numbers: " + industry + " " + geography, rot = 90, height = 400, width = 400).opts(shared_axes = False))

In [23]:
chart_creation = interact(create_chart, dataframe1 = fixed(df_labour),dataframe2 = fixed(df_business), geography=pn.widgets.Select(options=geography_list), industry=pn.widgets.Select(options=industry_list))

#Steps to serve up charts

In [24]:
to_display = pn.Column("#THESE ARE MY CHARTS",chart_creation)
to_display.servable()

Column
    [0] Markdown(str)
    [1] Column
        [0] Column
            [0] Select(options=['Ontario', 'Northwest Ter...], value='Ontario')
            [1] Select(options=['Educational services [61...], value='Educational s...)
        [1] Row
            [0] Row
                [0] HoloViews(Curve)
                [1] HoloViews(Curve)

In [25]:
print("Done")

Done


In [26]:
##WHAT DO THE VALUES MEAN? SHOULD THEY BE MULTIPLIED?

In [27]:
df_labour.to_csv("labour_out.csv")
df_business = df_business.loc[df_business["Business dynamics measure"]=="Active businesses"]
df_business.to_csv("business_out.csv")

#Try 3 by 5 layout of charts

In [28]:
# #This doesn't seem to work due to some bokeh or such issue


# print(geography_list2)

# def create_chart_bar(dataframe2, industry):
#     temp2 = dataframe2.loc[dataframe2["Industry"]==industry].copy()
#     temp2 = temp2.loc[temp2["Business dynamics measure"]=="Active businesses"]
#     chart2 = temp2.hvplot.bar(x="month_year", y = "VALUE",xlabel = "month_year", ylabel="VALUE", color = "red", title="This is title", rot = 90, height = 550, width = 550,subplots = True).opts(shared_axes = False)
#     return chart2
    


    
# # chart_creation_bar = interact(create_chart_bar, dataframe2 = fixed(df_business), industry=pn.widgets.Select(options=industry_list))

# # to_display2 = pn.Row("#THESE ARE MY CHARTS",chart_creation_bar)
# # to_display2.servable()

In [29]:
def create_chart_bar2(dataframe2, industry, geography):
    temp2 = dataframe2.loc[dataframe2["Industry"]==industry].copy()
    temp2 = temp2.loc[temp2["Business dynamics measure"]=="Active businesses"]
    chart2 = temp2.hvplot.bar(x="month_year", y = "VALUE",xlabel = "month_year", ylabel="VALUE", rot = 90, height = 550, width = 550,subplots = True, by="GEO").opts(shared_axes = False)
    return chart2

chart_creation_bar = interact(create_chart_bar2, dataframe2 = fixed(df_business),geography = fixed(geography_list2), industry=pn.widgets.Select(options=industry_list, ))

to_display2 = pn.Row(chart_creation_bar)
to_display2.servable()

Row
    [0] Column
        [0] Column
            [0] Select(options=['Educational services [61...], value='Educational s...)
        [1] Row
            [0] HoloViews(NdLayout, name='interactive02011')

In [30]:
temp = df_business.loc[df_business["Business dynamics measure"]=="Active businesses"]
temp.hvplot.line(x="month_year", y = "VALUE",xlabel = "month_year", ylabel="VALUE", rot = 90, height = 550, width = 550, groupby=["Industry","GEO"],subplots=True).overlay("GEO") 

:DynamicMap   [Industry]
   :NdOverlay   [GEO]
      :Curve   [month_year]   (VALUE)

In [31]:
new = google_lookup("restaurants")

In [32]:
new.hvplot.line(subplots=True)

:NdLayout   [Variable]
   :Curve   [date]   (value)

In [33]:
temp.reset_index(inplace = True)

In [34]:
# fig = make_subplots(rows=5, cols=3)


df_business = df_business.loc[df_business["Business dynamics measure"]=="Active businesses"]
df_business.head()
df_business_pivot = df_business.pivot_table(index= [df_business.index, "Industry"], columns = "GEO", values = ["VALUE"])
df_business_pivot.columns = df_business_pivot.columns.droplevel()
df_business_pivot.to_csv("pivot.csv")

def normalized(table, geography, industry):
    temp2 = table.xs(industry, level=1, drop_level=False)
    return temp2.hvplot.bar(x="month_year", y = geography,xlabel = "month_year", ylabel="VALUE", rot = 90, height = 550, width = 550,subplots = True).opts(shared_axes = False)
normalized_bar = interact(normalized, table = fixed(df_business_pivot),geography = fixed(geography_list2), industry=pn.widgets.Select(options=industry_list))

ValueError: Unexpected option 'logx' for NdLayout type across all extensions. No similar options found.

In [ ]:
# fig = py.subplots.make_subplots(rows=14, cols=1, subplot_titles=df.columns)
# j = 1
# for i in df_business.columns:
#     fig.add_trace(
#         go.Scatter(
#             {'x': df_business.index, 
#              'y': df_business[i]}), 
#              row=j, col=1)
#     j += 1